# Calculating polymer-polymer interactions

Demo how to calculate polymer-polymer interactions mapped onto polymer residues (fingerprints).

In [1]:
from pyspark.sql import SparkSession                  
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.interactions import InteractionFilter, InteractionFingerprinter                                 

#### Configure Spark 

In [2]:
spark = SparkSession.builder.master("local[4]").appName("PolymerInteractionFingerprintDemo").getOrCreate()

## Download PDB structure 1OHR

In [3]:
pdb = mmtfReader.download_mmtf_files(['1OHR'])

## Find ASP-ARG salt bridges among polymer chains

In [4]:
interactionFilter = InteractionFilter(distanceCutoff=3.5, minInteractions=1)
interactionFilter.set_query_groups(True, "ASP")
interactionFilter.set_query_atom_names(True, ['OD1','OD2'])
interactionFilter.set_target_groups(True, "ARG")
interactionFilter.set_target_atom_names(True, ['NH1','NH2'])

interactions = InteractionFingerprinter.get_polymer_interactions(pdb, interactionFilter)

This structure has two chains A and B. The target group (ARG 8) forms a salt bridge with the query group (ASP). Group numbers are the residue numbers of the interacting groups and sequence indices are zero-based indices of the interacting groups in the polymer sequence.

In [5]:
interactions.toPandas().head()    

,structureChainId,queryChainId,targetChainId,groupNumbers,sequenceIndices,sequence
0,1OHR.A,B,A,[8],[7],PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
1,1OHR.B,A,B,[8],[7],PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...


## Terminate Spark

In [6]:
spark.stop()